# MED-PC Extracting the Recording Data and Metadata

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
from datetime import datetime
from IPython.display import Video

In [2]:
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

## Getting the Metadata from all the MED-PC Recording Files

- Getting all the file paths of the recording files(that happen to all end in `.txt`

# NOTE: The following path must be changed to the directory where your MED-PC recording files are located, if they are not in the specied folder

In [5]:
all_med_pc_file = glob.glob("./data/recording_with_visual_cue/*.txt")

- Use this instead if you're using you're own data

In [6]:
all_med_pc_file[:10]

['./data/recording_with_visual_cue/2022-07-01_12h47m_Subject .txt',
 './data/recording_with_visual_cue/2022-07-01_12h54m_Subject .txt']

- Example of what the MED-PC Recording file looks like

In [7]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-07-01_12h47m_Subject .txt







Start Date: 07/01/22

End Date: 07/01/22

Subject: 

Experiment: 

Group: 

Box: 1

Start Time: 12:47:43

End Time: 12:53:52

MSN: iwata_clock_test

A:    3399.000

D:    7500.000

F:    2000.000

L:       0.000

O:       0.000

T:     344.020

U:     310.010



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```
    
- We will just find all the lines that start with "File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN". And then stop once all the metadata types have been collected

In [8]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [9]:
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/recording_with_visual_cue/2022-07-01_12h47m_Subject .txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-07-01_12h47m_Subject .txt', 'Start Date': '07/01/22', 'End Date': '07/01/22', 'Subject': '', 'Experiment': '', 'Group': '', 'Box': '1', 'Start Time': '12:47:43', 'End Time': '12:53:52', 'MSN': 'iwata_clock_test'}


## Making a Dataframe out of the Metadata

In [10]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [11]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/recording_with_visual_cue/2022-07-01_12...,C:\MED-PC\Data\2022-07-01_12h47m_Subject .txt,07/01/22,07/01/22,,,,1,12:47:43,12:53:52,iwata_clock_test
1,./data/recording_with_visual_cue/2022-07-01_12...,C:\MED-PC\Data\2022-07-01_12h54m_Subject .txt,07/01/22,07/01/22,,,,1,12:54:17,13:00:09,iwata_clock_test


In [12]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/recording_with_visual_cue/2022-07-01_12...,C:\MED-PC\Data\2022-07-01_12h47m_Subject .txt,07/01/22,07/01/22,,,,1,12:47:43,12:53:52,iwata_clock_test
1,./data/recording_with_visual_cue/2022-07-01_12...,C:\MED-PC\Data\2022-07-01_12h54m_Subject .txt,07/01/22,07/01/22,,,,1,12:54:17,13:00:09,iwata_clock_test


- Just getting the numbers out of the column that contains the cage information

## Inputting all the MED-PC log files

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [13]:
with open("./scripts/levelNP_CS_reward_laserepochON1st_noshock.MPC") as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\v3 stop tone with poke

\v3.2 monitor port entries AND exits



\INPUTS

^port = 8



\OUTPUTS

^fan = 16

^houselight = 11

^tone1 = 2

^tone2 = 3

^tone3 = 4

^tone4 = 5

^pump = 9

^whitenoise = 1

^laser=10

^csout = 5

^peout = 15

^cs1out = 12

^cs2out = 13

^cs3out = 14



\EXP SETTINGS

^ncsNoShock = 0

^initCS1trials = 3



\ARRAYS

DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity



LIST V = 30", 30", 30", 30", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 120", 80", 95", 80", 90", 80", 75", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 90", 80", 90", 100", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90"

- We will be using the comments in the MED-PC script(which starts with the `\`) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable.
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the comments in the MED-PC script that we will use the names from:

```
\Variables

\A - Time since last CS

\B - Shock intensity

\C - Counter array

\D - Current ITI value

\E - CS ITI values for first few trials

\F - Shock duration

\G -

\H -

\I -

\J - Shock intensity repo

\K - CS type

\L -

\M - CS type repo

\N - Port exit time stamp array

\O -

\P - Port entry time stamp array

\Q - Sucrose delivery time stamp array (absolute)

\R - Sucrose delivery time stamp array (relative to last CS)

\S - CS presentation time stamp array

\T - Session timer

\U - Time since last CS presentation

\V - List of CS ITI values (tone + houselight)

\W - ITI values used for CS one each trial

\X -

\Y - Beam break monitor variable

\Z -
```

In [14]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-07-01_12h47m_Subject .txt







Start Date: 07/01/22

End Date: 07/01/22

Subject: 

Experiment: 

Group: 

Box: 1

Start Time: 12:47:43

End Time: 12:53:52

MSN: iwata_clock_test

A:    3399.000

D:    7500.000

F:    2000.000

L:       0.000

O:       0.000

T:     344.020

U:     310.010



- **Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder**

In [15]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)
concatted_medpc_df = concatted_medpc_df.reset_index(drop=True)

In [16]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,253.30,64.0,399.0,0.0,60.01,8.0,49.0,12.0,254.27,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
1,255.08,314.0,399.0,0.0,140.01,28.0,50.0,12.0,255.79,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
2,256.75,NaN,NaN,0.0,230.01,58.0,51.0,12.0,257.63,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
3,258.44,NaN,NaN,0.0,310.01,18.0,53.0,12.0,259.22,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
4,259.80,NaN,NaN,0.0,8000.00,0.0,0.0,0.0,260.50,0.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...


- Dropping all rows that don't have the CS presentation time

In [17]:
cs_time_df = concatted_medpc_df[concatted_medpc_df["(G)CS_secondspresentation"] != 0.0].dropna(subset="(G)CS_secondspresentation")

In [18]:
cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,253.30,64.0,399.0,0.0,60.01,8.0,49.0,12.0,254.27,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
1,255.08,314.0,399.0,0.0,140.01,28.0,50.0,12.0,255.79,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
2,256.75,NaN,NaN,0.0,230.01,58.0,51.0,12.0,257.63,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
3,258.44,NaN,NaN,0.0,310.01,18.0,53.0,12.0,259.22,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
2541,75.05,64.0,399.0,0.0,60.01,21.0,55.0,12.0,75.75,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
2542,76.20,144.0,399.0,0.0,140.01,41.0,56.0,12.0,76.82,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
2543,77.65,234.0,399.0,0.0,230.01,12.0,58.0,12.0,78.32,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...
2544,78.94,314.0,399.0,0.0,310.01,31.0,59.0,12.0,79.71,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...


# Making a column for the time

- Because the hour, the minute, and the seconds are all in different columns we will combine them into one

In [19]:
cs_time_df.columns

Index(['(P)Portentry', '(Q)USdelivery', '(R)UStime', '(W)ITIvalues',
       '(S)CSpresentation', '(G)CS_secondspresentation',
       '(H)CS_minutespresentation', '(I)CS_hourspresentation', '(N)Portexit',
       '(K)CStype', '(B)shockintensity', 'date', 'subject', 'file_path'],
      dtype='object')

- Making the numbers for hours, minutes, and seconds into whole numbers

In [20]:
cs_time_df["cs_second"] = cs_time_df["(G)CS_secondspresentation"].astype(int).astype(str)
cs_time_df["cs_minute"] = cs_time_df["(H)CS_minutespresentation"].astype(int).astype(str)
cs_time_df["cs_hour"] = cs_time_df["(I)CS_hourspresentation"].astype(int).astype(str)


- Combining the hours, minutes, and seconds into one column as a string

In [21]:
cs_time_df["cs_time_str"] = cs_time_df.apply(lambda x: ":".join([x["cs_hour"], x["cs_minute"],  x["cs_second"]]), axis=1)

- Converting the time into a Python datetime object
    - This will allow us to make calculations

In [22]:
cs_time_df["cs_time_object"] = cs_time_df["cs_time_str"].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))


In [23]:
cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path,cs_second,cs_minute,cs_hour,cs_time_str,cs_time_object
0,253.30,64.0,399.0,0.0,60.01,8.0,49.0,12.0,254.27,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,8,49,12,12:49:8,1900-01-01 12:49:08
1,255.08,314.0,399.0,0.0,140.01,28.0,50.0,12.0,255.79,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,28,50,12,12:50:28,1900-01-01 12:50:28
2,256.75,NaN,NaN,0.0,230.01,58.0,51.0,12.0,257.63,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,58,51,12,12:51:58,1900-01-01 12:51:58
3,258.44,NaN,NaN,0.0,310.01,18.0,53.0,12.0,259.22,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,18,53,12,12:53:18,1900-01-01 12:53:18
2541,75.05,64.0,399.0,0.0,60.01,21.0,55.0,12.0,75.75,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,21,55,12,12:55:21,1900-01-01 12:55:21
2542,76.20,144.0,399.0,0.0,140.01,41.0,56.0,12.0,76.82,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,41,56,12,12:56:41,1900-01-01 12:56:41
2543,77.65,234.0,399.0,0.0,230.01,12.0,58.0,12.0,78.32,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,12,58,12,12:58:12,1900-01-01 12:58:12
2544,78.94,314.0,399.0,0.0,310.01,31.0,59.0,12.0,79.71,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,31,59,12,12:59:31,1900-01-01 12:59:31


# Importing Video Data

- These CSVs are create in Bonsai when the video is being recorded. They give a time stamp of when the video is first being recorded.

In [24]:
video_recording_time_stamps = pd.read_csv("./data/recording_with_visual_cue/clock_test_time_2022-07-01T12_48_15.csv", header=None)

In [25]:
video_recording_time_stamps.head()

,0,1
0,(7/1/2022 4:49:00 PM,984); 2022-07-01 12:49:00.951000
1,(7/1/2022 4:49:01 PM,16); 2022-07-01 12:49:00.981000
2,(7/1/2022 4:49:01 PM,48); 2022-07-01 12:49:01.013000
3,(7/1/2022 4:49:01 PM,81); 2022-07-01 12:49:01.046000
4,(7/1/2022 4:49:01 PM,112); 2022-07-01 12:49:01.077000


- The video is first being recorded in the time specified by the cell in the first row and the first column
    - We will try to get on the hour, minute, seconds from this
        - By getting everthing after the space, and then removing everything before the period

In [26]:
video_recording_time_stamps["time"] = video_recording_time_stamps[1].apply(lambda x: x.split()[-1].split(".")[0])


In [27]:
video_recording_time_stamps.head()

,0,1,time
0,(7/1/2022 4:49:00 PM,984); 2022-07-01 12:49:00.951000,12:49:00
1,(7/1/2022 4:49:01 PM,16); 2022-07-01 12:49:00.981000,12:49:00
2,(7/1/2022 4:49:01 PM,48); 2022-07-01 12:49:01.013000,12:49:01
3,(7/1/2022 4:49:01 PM,81); 2022-07-01 12:49:01.046000,12:49:01
4,(7/1/2022 4:49:01 PM,112); 2022-07-01 12:49:01.077000,12:49:01


- Getting the first time that's listed, which is when the video starts being recorded

In [28]:
video_recording_start_time_str = video_recording_time_stamps["time"][0]


In [29]:
video_recording_start_time_str

'12:49:00'

- Converting this into a datetime object so that we can make calculations

In [30]:
video_recording_start_time_object = datetime.strptime(video_recording_start_time_str, '%H:%M:%S')


In [31]:
video_recording_start_time_object

datetime.datetime(1900, 1, 1, 12, 49)

# Syncing the MED-PC timestamps with the time in the video

- To get when in the video something happens(i.e. 10 seconds), we will calculate the difference of the time of the event with the time that the video first starts being recorded
- NOTE: Make sure that you are using the associated files for the MED-PC recording file and the video file

In [32]:
first_medpc_recording_file = cs_time_df["file_path"].unique()[0]

first_cs_time_df = cs_time_df[cs_time_df["file_path"] == first_medpc_recording_file]

In [33]:
# The time of the first controlled stimulus 
first_cs_time_df["cs_time_object"].iloc[0]

Timestamp('1900-01-01 12:49:08')

In [34]:
# The time of the second controlled stiumuls
first_cs_time_df["cs_time_object"].iloc[1]

Timestamp('1900-01-01 12:50:28')

In [35]:
# The time that the video started being recorded
video_recording_start_time_object

datetime.datetime(1900, 1, 1, 12, 49)

In [36]:
# The time in the video that the controlled stimulus occurs
cs_01_time_object = first_cs_time_df["cs_time_object"].iloc[0] - video_recording_start_time_object 
cs_02_time_object = first_cs_time_df["cs_time_object"].iloc[1] - video_recording_start_time_object 
cs_03_time_object = first_cs_time_df["cs_time_object"].iloc[2] - video_recording_start_time_object 

In [37]:
cs_01_time_object.seconds

8

In [38]:
cs_02_time_object.seconds

88

In [39]:
cs_03_time_object.seconds

178

# Trimming the video

In [40]:
video_object = VideoFileClip("./data/recording_with_visual_cue/clock_test_recording_2022-07-01T12_49_00.avi")

In [41]:
video_object.duration

293.7

fixed func issue with: https://github.com/Zulko/moviepy/issues/1765


In [42]:
cs1_trimmed_video = video_object.subclip(cs_01_time_object.seconds, cs_02_time_object.seconds + 20)

In [43]:
cs2_trimmed_video = video_object.subclip(cs_02_time_object.seconds, cs_03_time_object.seconds + 20)

In [44]:
cs3_trimmed_video = video_object.subclip(cs_03_time_object.seconds, video_object.duration)

In [45]:
cs1_trimmed_video.to_videofile("./data/recording_with_visual_cue/trimmed_cs1.mp4")

Moviepy - Building video ./data/recording_with_visual_cue/trimmed_cs1.mp4.
Moviepy - Writing video ./data/recording_with_visual_cue/trimmed_cs1.mp4



Moviepy - Done !
Moviepy - video ready ./data/recording_with_visual_cue/trimmed_cs1.mp4


In [46]:
cs2_trimmed_video.to_videofile("./data/recording_with_visual_cue/trimmed_cs2.mp4")

Moviepy - Building video ./data/recording_with_visual_cue/trimmed_cs2.mp4.
Moviepy - Writing video ./data/recording_with_visual_cue/trimmed_cs2.mp4



Moviepy - Done !
Moviepy - video ready ./data/recording_with_visual_cue/trimmed_cs2.mp4


In [47]:
cs3_trimmed_video.to_videofile("./data/recording_with_visual_cue/trimmed_cs3.mp4")

Moviepy - Building video ./data/recording_with_visual_cue/trimmed_cs3.mp4.
Moviepy - Writing video ./data/recording_with_visual_cue/trimmed_cs3.mp4



Moviepy - Done !
Moviepy - video ready ./data/recording_with_visual_cue/trimmed_cs3.mp4


In [48]:
Video("./data/recording_with_visual_cue/trimmed_cs1.mp4")

# Repeating again with a different video

# Importing Video Data

- These CSVs are create in Bonsai when the video is being recorded. They give a time stamp of when the video is first being recorded.

In [49]:
video_recording_time_stamps = pd.read_csv("./data/recording_with_visual_cue/clock_test_time_2022-07-01T12_54_07.csv", header=None)

In [50]:
video_recording_time_stamps.head()

,0,1
0,(7/1/2022 4:58:56 PM,338); 2022-07-01 12:58:56.230000
1,(7/1/2022 4:58:56 PM,370); 2022-07-01 12:58:56.260000
2,(7/1/2022 4:58:56 PM,418); 2022-07-01 12:58:56.308000
3,(7/1/2022 4:58:56 PM,450); 2022-07-01 12:58:56.340000
4,(7/1/2022 4:58:56 PM,482); 2022-07-01 12:58:56.372000


- The video is first being recorded in the time specified by the cell in the first row and the first column
    - We will try to get on the hour, minute, seconds from this
        - By getting everthing after the space, and then removing everything before the period

In [51]:
video_recording_time_stamps["time"] = video_recording_time_stamps[1].apply(lambda x: x.split()[-1].split(".")[0])


In [52]:
video_recording_time_stamps.head()

,0,1,time
0,(7/1/2022 4:58:56 PM,338); 2022-07-01 12:58:56.230000,12:58:56
1,(7/1/2022 4:58:56 PM,370); 2022-07-01 12:58:56.260000,12:58:56
2,(7/1/2022 4:58:56 PM,418); 2022-07-01 12:58:56.308000,12:58:56
3,(7/1/2022 4:58:56 PM,450); 2022-07-01 12:58:56.340000,12:58:56
4,(7/1/2022 4:58:56 PM,482); 2022-07-01 12:58:56.372000,12:58:56


- Getting the first time that's listed, which is when the video starts being recorded

In [53]:
video_recording_start_time_str = video_recording_time_stamps["time"][0]


In [54]:
video_recording_start_time_str

'12:58:56'

- Converting this into a datetime object so that we can make calculations

In [55]:
video_recording_start_time_object = datetime.strptime(video_recording_start_time_str, '%H:%M:%S')


In [56]:
video_recording_start_time_object

datetime.datetime(1900, 1, 1, 12, 58, 56)

# Syncing the MED-PC timestamps with the time in the video

- To get when in the video something happens(i.e. 10 seconds), we will calculate the difference of the time of the event with the time that the video first starts being recorded
- NOTE: Make sure that you are using the associated files for the MED-PC recording file and the video file

In [57]:
second_medpc_recording_file = cs_time_df["file_path"].unique()[1]

second_cs_time_df = cs_time_df[cs_time_df["file_path"] == second_medpc_recording_file]

In [68]:
second_cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path,cs_second,cs_minute,cs_hour,cs_time_str,cs_time_object
2541,75.05,64.0,399.0,0.0,60.01,21.0,55.0,12.0,75.75,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,21,55,12,12:55:21,1900-01-01 12:55:21
2542,76.20,144.0,399.0,0.0,140.01,41.0,56.0,12.0,76.82,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,41,56,12,12:56:41,1900-01-01 12:56:41
2543,77.65,234.0,399.0,0.0,230.01,12.0,58.0,12.0,78.32,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,12,58,12,12:58:12,1900-01-01 12:58:12
2544,78.94,314.0,399.0,0.0,310.01,31.0,59.0,12.0,79.71,1.0,0.0,20220701,,./data/recording_with_visual_cue/2022-07-01_12...,31,59,12,12:59:31,1900-01-01 12:59:31


In [70]:
# The time of the first controlled stimulus 
second_cs_time_df["cs_time_object"].iloc[3]

Timestamp('1900-01-01 12:59:31')

In [71]:
# The time that the video started being recorded
video_recording_start_time_object

datetime.datetime(1900, 1, 1, 12, 58, 56)

In [72]:
# The time in the video that the controlled stimulus occurs
# cs_01_time_object = second_cs_time_df["cs_time_object"].iloc[0] - video_recording_start_time_object 
# cs_02_time_object = second_cs_time_df["cs_time_object"].iloc[1] - video_recording_start_time_object 
# cs_03_time_object = second_cs_time_df["cs_time_object"].iloc[2] - video_recording_start_time_object 
cs_04_time_object = second_cs_time_df["cs_time_object"].iloc[3] - video_recording_start_time_object 


In [73]:
cs_04_time_object.seconds

35

# Trimming the video

In [74]:
video_object = VideoFileClip("./data/recording_with_visual_cue/clock_test_recording_2022-07-01T12_58_56.avi")

In [75]:
video_object.duration

77.5

fixed func issue with: https://github.com/Zulko/moviepy/issues/1765


In [76]:
cs1_trimmed_video = video_object.subclip(cs_04_time_object.seconds, video_object.duration)

In [78]:
cs1_trimmed_video.to_videofile("./data/recording_with_visual_cue/trimmed_cs4.mp4")

Moviepy - Building video ./data/recording_with_visual_cue/trimmed_cs4.mp4.
Moviepy - Writing video ./data/recording_with_visual_cue/trimmed_cs4.mp4



Moviepy - Done !
Moviepy - video ready ./data/recording_with_visual_cue/trimmed_cs4.mp4


In [79]:
Video("./data/recording_with_visual_cue/trimmed_cs4.mp4")